In [1]:
# !pip -q --no-cache-dir install git+https://github.com/hcss-utils/lensorg.git@refactoring 
!pip -q --no-cache-dir install git+https://github.com/hcss-utils/lensorg.git
!pip -q --no-cache-dir install ratelimit python-dotenv

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd
from lensorg import chunks, remote_call, process, token

In [4]:
# if running from google colab/not using dotenv, paste token after 'or'
token = token or ""

In [5]:
# loading data
df = pd.read_csv("/content/drive/MyDrive/RuBase/Bibliometrics/Deterrence/December 2020/Scite/deterrence-broad-scite.csv")

# selecting valid DOIs
DOIs = df.loc[df["doi"].str.contains("^10\.\d{4,9}/[-._;()/:a-zA-Z0-9]+$"), "doi"].tolist()

In [6]:
# first 5 DOIs
DOIs[:5]

['10.1016/0304-405x(94)00823-j',
 '10.1017/s0033291714000129',
 '10.1126/science.1163732',
 '10.1046/j.1365-2125.2001.01306.x',
 '10.1017/cbo9780511897948']

In [7]:
print(f"we have {len(DOIs)} DOIs, but each request could only take 1000 DOIs")

we have 18406 DOIs, but each request could only take 1000 DOIs


In [9]:
container = []
# testing on 2_000 DOIs
for chunk in chunks(DOIs[:2_000], n=999):
    response = remote_call(dois=chunk, token=token)
    data = process(response)
    container.append(
        pd.DataFrame(data)
    )

In [10]:
result = pd.concat(container, ignore_index=True)

In [11]:
result.shape

(1997, 2)

In [12]:
result.head()

,doi,fields
0,10.1007/s004420050815,Nutrient_Food choice_Adenostyles alliariae_Pet...
1,10.1111/j.1540-5915.2012.00361.x,Business_Marketing_Employee research_Organisat...
2,10.1046/j.1365-2125.1997.00616.x,Public health_Pediatrics_Alternative medicine_...
3,10.1016/j.socscimed.2008.12.031,Psychology_Health care_Internal audit_Internal...
4,10.1080/03235408.2013.858879,Horticulture_Lepidoptera genitalia_PEST analys...
